In [2]:
# linear modelling functions
#HENRY AND JUN
#MERGED KAMMEN + COOLCLIMAE
# !pip install ipynb
#import ipynb.fs.full.general_model_pipeline_vl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

## Forecasting and Prediction Modeling (25 points)

This section is where the rubber meets the road.  In it you must:
1. Explore at least 3 prediction modeling approaches for each prediction question, ranging from the simple (e.g. linear regression, KNN) to the complex (e.g. SVM, random forests, Lasso).  
2. Motivate all your modeling decisions.  This includes parameter choices (e.g., how many folds in k-fold cross validation, what time window you use for averaging your data) as well as model form (e.g., If you use regression trees, why?  If you include nonlinear features in a regression model, why?). 
1. Carefully describe your cross validation and model selection process.  You should partition your data into training and testing data sets.  The training data set is what you use for cross-validation (i.e. you sample from within it to create folds, etc.).  The testing data set is held to the very end of your efforts, and used to compare qualitatively different models (e.g. OLS vs random forests).
4. Very carefully document your workflow.  We will be reading a lot of projects, so we need you to explain each basic step in your analysis.  
5. Seek opportunities to write functions allow you to avoid doing things over and over, and that make your code more succinct and readable. 

In [9]:
df = pd.read_csv('../../data/new-cleaned-household-data.csv')
df

,input_income,input_footprint_transportation_num_vehicles,input_footprint_transportation_mpg1,input_footprint_transportation_miles1,input_footprint_transportation_fuel1,input_footprint_transportation_airshort,input_footprint_transportation_airmedium,input_footprint_transportation_airlong,input_footprint_housing_cdd,input_footprint_housing_hdd,input_footprint_housing_electricity_dollars,input_footprint_housing_squarefeet,input_footprint_housing_watersewage,input_footprint_shopping_food_meatfisheggs_default,input_footprint_shopping_goods_clothing,input_footprint_shopping_goods_other_personalcare,input_footprint_shopping_services_education,input_footprint_shopping_services_total,result_grand_total
0,1.0,1.0,22.0,8700.0,1.0,2.0,0.0,0.0,1085.39,1455.31,470.0,1591.0,237.00,543.0,169.350149,47.227769,55.002939,963.524289,18.998752
1,1.0,1.0,22.0,8500.0,1.0,2.0,0.0,0.0,1085.39,1455.31,510.0,1591.0,237.00,543.0,161.234333,45.486592,51.436731,935.303780,18.960587
2,1.0,1.0,22.0,8000.0,1.0,2.0,0.0,0.0,1085.39,1455.31,490.0,1591.0,237.00,543.0,157.543718,44.716724,49.867708,923.983913,18.488686
3,1.0,1.0,22.0,7300.0,1.0,1.0,1.0,0.0,1085.39,1455.31,360.0,1591.0,237.00,543.0,133.502127,42.053263,45.315034,1012.205374,18.001409
4,1.0,1.0,22.0,6000.0,1.0,4.0,0.0,0.0,1085.39,1455.31,310.0,1591.0,237.00,543.0,121.236910,38.452350,37.588628,902.778103,16.311888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163,11.0,2.0,22.0,13300.0,1.0,13.0,2.0,0.0,1079.48,4055.28,680.0,2500.0,427.12,543.0,332.387392,80.597450,93.997996,3125.040824,44.967169
1164,11.0,2.0,22.0,13100.0,1.0,0.0,0.0,2.0,1079.48,4055.28,670.0,2500.0,427.12,543.0,298.647375,72.419332,74.865184,2698.769056,42.249874
1165,11.0,2.0,22.0,14700.0,1.0,4.0,2.0,1.0,1079.48,4055.28,660.0,2500.0,427.12,543.0,340.838237,80.989287,94.500956,3062.069816,45.462417
1166,11.0,2.0,22.0,15400.0,1.0,8.0,0.0,1.0,1079.48,4055.28,800.0,2500.0,427.12,543.0,361.562873,82.532223,97.241368,2967.253415,47.138023


In [13]:
X = df[['input_income', 'input_footprint_transportation_num_vehicles', 'input_footprint_transportation_mpg1', 'input_footprint_transportation_miles1', 'input_footprint_transportation_fuel1', 'input_footprint_transportation_airlong', 'input_footprint_housing_cdd', 'input_footprint_housing_hdd', 'input_footprint_housing_electricity_dollars', 'input_footprint_housing_squarefeet', 'input_footprint_housing_watersewage', 'input_footprint_shopping_food_meatfisheggs_default', 'input_footprint_shopping_goods_clothing', 'input_footprint_shopping_goods_other_personalcare', 'input_footprint_shopping_services_education', 'input_footprint_shopping_services_total']]
y = df['result_grand_total']


In [14]:
X_train, X_test, y_train, y_test = train_test_split(newX, newy, test_size=0.25, random_state = 2020)

X_train, X_test, y_train, y_test

lm = LinearRegression()
lm.fit(X_train, y_train)
predictions = lm.predict(X_test)

print('R^2: ', r2_score(y_test, predictions))
print('MSE: ', mean_squared_error(y_test, predictions))
    

R^2:  0.9947588200959437
MSE:  0.6115182447836953


## Interpretation and Conclusions (20 points)
In this section you must relate your modeling and forecasting results to your original prediction question.  You must:
1. Address a resource allocation question.  What do the answers mean? What advice would you give a decision maker on the basis of your results?  How might they allocate their resources differently with the results of your model?  Why should the reader care about your results?
2. Discuss caveats and / or reasons your results might be flawed.  No model is perfect, and understanding a model's imperfections is extremely important for the purpose of knowing how to interpret your results.  Often, we know the model output is wrong but we can assign a direction for its bias.  This helps to understand whether or not your answers are conservative.  

Shoot for 500-1000 words for this section.